In [ ]:
import cv2
import numpy as np
import math
import time
import matplotlib.pyplot as plt
import os
import random

OUTPUT_FOLDER = "tracking_results"
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# PART 1: MATH & PLOTTING

def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def from_scratch_derivatives(position_track, fps):
    if len(position_track) < 5:
        return (np.array([]), np.array([]), np.array([]), np.array([]))
        
    dt = 1.0 / fps
    positions = np.array(position_track)
    
    velocity = np.diff(positions, axis=0) / dt
    speed = np.linalg.norm(velocity, axis=1)
    
    acceleration_vec = np.diff(velocity, axis=0) / dt
    acceleration = np.linalg.norm(acceleration_vec, axis=1)
    
    jerk_vec = np.diff(acceleration_vec, axis=0) / dt
    jerk = np.linalg.norm(jerk_vec, axis=1)
    
    jounce_vec = np.diff(jerk_vec, axis=0) / dt
    jounce = np.linalg.norm(jounce_vec, axis=1)
    
    return speed, acceleration, jerk, jounce

def save_derivative_plot(track_id, speed, acceleration, jerk, jounce, fps):
    if speed.size == 0: return
    
    time_speed = np.arange(len(speed)) / fps
    time_acc = np.arange(len(acceleration)) / fps
    time_jerk = np.arange(len(jerk)) / fps
    time_jounce = np.arange(len(jounce)) / fps
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle(f'Motion Derivatives for Track ID: {track_id}', fontsize=16)
    
    ax1.plot(time_speed, speed, 'b-', label='Speed')
    ax1.set_title('Speed')
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(time_acc, acceleration, 'r-', label='Acceleration')
    ax2.set_title('Acceleration')
    ax2.grid(True, alpha=0.3)
    
    ax3.plot(time_jerk, jerk, 'g-', label='Jerk')
    ax3.set_title('Jerk')
    ax3.grid(True, alpha=0.3)
    
    ax4.plot(time_jounce, jounce, 'm-', label='Jounce')
    ax4.set_title('Jounce')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    filename = f"{OUTPUT_FOLDER}/track_{track_id}.png"
    plt.savefig(filename)
    plt.close(fig)
    print(f"Saved graph for Track {track_id} to {filename}")

# PART 2: SIMPLE DBSCAN + ROBUST TRACKER

def dbscan_numpy(points, eps=10, min_samples=3):
    """Simple DBSCAN with RELAXED parameters for better detection"""
    if points.size == 0:
        return np.array([], dtype=int)
    
    N = points.shape[0]
    labels = -1 * np.ones(N, dtype=int)
    visited = np.zeros(N, dtype=bool)
    cluster_id = 0

    for i in range(N):
        if visited[i]:
            continue
        visited[i] = True

        diffs = points - points[i]
        dists2 = np.einsum('ij,ij->i', diffs, diffs)
        neighbors = np.where(dists2 <= eps * eps)[0]

        if neighbors.size < min_samples:
            labels[i] = -1
        else:
            labels[i] = cluster_id
            seeds = list(neighbors[neighbors != i].tolist())
            while seeds:
                current = seeds.pop()
                if not visited[current]:
                    visited[current] = True
                    diffs_c = points - points[current]
                    dists2_c = np.einsum('ij,ij->i', diffs_c, diffs_c)
                    nbrs_c = np.where(dists2_c <= eps * eps)[0]
                    if nbrs_c.size >= min_samples:
                        for n in nbrs_c:
                            if n not in seeds:
                                seeds.append(n)
                if labels[current] == -1:
                    labels[current] = cluster_id
            cluster_id += 1
    return labels

class RobustTracker:
    def __init__(self, blur_ksize, threshold_val, min_contour_area, max_track_distance, patience, dbscan_eps=10, dbscan_minpts=3, dbscan_sample_limit=5000):
        self.blur_ksize = blur_ksize
        self.threshold_val = threshold_val
        self.min_contour_area = min_contour_area
        self.max_track_distance = max_track_distance
        self.patience_limit = patience
        
        self.prev_frame_gray = None
        self.tracks = {} 
        self.track_patience = {} 
        self.next_track_id = 0
        self.frame_number = 0

        # RELAXED DBSCAN params
        self.dbscan_eps = dbscan_eps
        self.dbscan_minpts = dbscan_minpts
        self.dbscan_sample_limit = dbscan_sample_limit

    def _preprocess_frame(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, self.blur_ksize, 0)
        return blurred

    def _update_tracks(self, current_centroids):
        self.frame_number += 1
        
        if not self.tracks:
            for centroid in current_centroids:
                self.tracks[self.next_track_id] = [centroid]
                self.track_patience[self.next_track_id] = self.patience_limit
                self.next_track_id += 1
            return

        active_track_ids = list(self.tracks.keys())
        last_positions = [self.tracks[tid][-1] for tid in active_track_ids]
        
        matched_new_centroids = set()
        matched_track_ids = set()

        for idx, track_id in enumerate(active_track_ids):
            last_pos = last_positions[idx]
            
            best_dist = float('inf')
            best_centroid_idx = -1
            
            for i, centroid in enumerate(current_centroids):
                if i in matched_new_centroids:
                    continue
                
                dist = euclidean_distance(last_pos, centroid)
                if dist < best_dist:
                    best_dist = dist
                    best_centroid_idx = i
            
            if best_dist < self.max_track_distance and best_centroid_idx != -1:
                self.tracks[track_id].append(current_centroids[best_centroid_idx])
                self.track_patience[track_id] = self.patience_limit
                
                matched_new_centroids.add(best_centroid_idx)
                matched_track_ids.add(track_id)
        
        for track_id in active_track_ids:
            if track_id not in matched_track_ids:
                if track_id not in self.track_patience:
                     self.track_patience[track_id] = 0
                self.track_patience[track_id] -= 1

        for i, centroid in enumerate(current_centroids):
            if i not in matched_new_centroids:
                self.tracks[self.next_track_id] = [centroid]
                self.track_patience[self.next_track_id] = self.patience_limit
                self.next_track_id += 1

    def _draw_overlay(self, frame, num_objects, processing_time):
        cv2.putText(frame, f"Objects: {num_objects}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        for track_id, positions in self.tracks.items():
            if self.track_patience.get(track_id, 0) > 0:
                last_pos = positions[-1]
                cv2.circle(frame, last_pos, 6, (0, 0, 255), -1)
                cv2.putText(frame, f"ID:{track_id}", (last_pos[0]+10, last_pos[1]), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        return frame

    def _detect_objects_via_dbscan(self, motion_mask):
        """Detect objects using DBSCAN clustering"""
        ys, xs = np.where(motion_mask > 0)
        if xs.size == 0:
            return []

        points = np.column_stack((xs, ys)).astype(np.float32)
        N = points.shape[0]
        
        # Sample if too many points
        if N > self.dbscan_sample_limit:
            idxs = np.random.choice(N, self.dbscan_sample_limit, replace=False)
            points_sampled = points[idxs]
            sampled_map = idxs
        else:
            points_sampled = points
            sampled_map = None

        labels_sampled = dbscan_numpy(points_sampled, eps=self.dbscan_eps, min_samples=self.dbscan_minpts)
        unique_labels = np.unique(labels_sampled)
        centroids = []

        for lbl in unique_labels:
            if lbl == -1:
                continue
            mask = labels_sampled == lbl
            cluster_points = points_sampled[mask]

            if sampled_map is not None:
                est_centroid = cluster_points.mean(axis=0)
                diffs_full = points - est_centroid
                d2_full = np.einsum('ij,ij->i', diffs_full, diffs_full)
                cluster_full_idx = np.where(d2_full <= (self.dbscan_eps * 1.5)**2)[0]
                area = cluster_full_idx.size
                if area < self.min_contour_area:
                    continue
                centroid = tuple(np.round(est_centroid).astype(int).tolist())
                centroids.append(centroid)
            else:
                area = cluster_points.shape[0]
                if area < self.min_contour_area:
                    continue
                centroid = tuple(np.round(cluster_points.mean(axis=0)).astype(int).tolist())
                centroids.append(centroid)

        return centroids

    def process_video(self, video_path, output_video_path):
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("Error opening video")
            return None, 0
        
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
        
        # initialize previous frame BEFORE loop
        ret, first_frame = cap.read()
        if not ret: 
            print("Cannot read first frame")
            return None, 0
        
        self.prev_frame_gray = self._preprocess_frame(first_frame)
        print(f"Video loaded: {width}x{height} @ {fps} fps")
        
        frame_count = 0
        while cap.isOpened():
            start_time = time.time()
            ret, frame = cap.read()
            if not ret: 
                break
            
            frame_count += 1
            
            # Preprocess
            current_frame_gray = self._preprocess_frame(frame)
            
            # Motion detection
            diff_frame = cv2.absdiff(current_frame_gray, self.prev_frame_gray)
            _, motion_mask = cv2.threshold(diff_frame, self.threshold_val, 255, cv2.THRESH_BINARY)
            
            # IMPROVED: Opening + Dilation
            kernel = np.ones((3, 3), np.uint8)
            motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel, iterations=1)
            motion_mask = cv2.dilate(motion_mask, kernel, iterations=2)
            
            # Detect objects
            current_centroids = self._detect_objects_via_dbscan(motion_mask)
            
            # Update tracks
            self._update_tracks(current_centroids)
            
            proc_ms = (time.time() - start_time) * 1000
            
            # Display
            display_frame = self._draw_overlay(frame, len(current_centroids), proc_ms)
            writer.write(display_frame)
            cv2.imshow("Robust Tracking (DBSCAN)", display_frame)
            
            self.prev_frame_gray = current_frame_gray
            
            if frame_count % 30 == 0:
                print(f"Processed {frame_count} frames, detected {len(current_centroids)} objects")
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        writer.release()
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        
        print(f"\nTotal frames processed: {frame_count}")
        print(f"Total tracks created: {self.next_track_id}")
        
        return self.tracks, fps

    def analyze_tracks_wrapper(self, fps):
        print("\n--- Post-Processing Analysis ---")
        count = 0
        for track_id, positions in self.tracks.items():
            if len(positions) < 30: 
                continue
            
            s, a, j, s4 = from_scratch_derivatives(positions, fps)
            save_derivative_plot(track_id, s, a, j, s4, fps)
            count += 1
        
        if count == 0:
            print("No long tracks found. Try adjusting parameters.")
        else:
            print(f"Analyzed {count} tracks")

if __name__ == "__main__":
    # RELAXED PARAMETERS for better detection
    tracker = RobustTracker(
        blur_ksize=(15, 15),
        threshold_val=20,           # LOWERED from 25
        min_contour_area=50,        # LOWERED from 200
        max_track_distance=150,
        patience=15,
        dbscan_eps=10,              # INCREASED from 4
        dbscan_minpts=3,            # LOWERED from 8
        dbscan_sample_limit=5000
    )
    
    tracks, video_fps = tracker.process_video('output.mp4', 'processed_video.mp4')
    
    if tracks:
        tracker.analyze_tracks_wrapper(video_fps)
    else:
        print("No tracks found. Check your video file.")